In [188]:
import numpy as np
import pandas as pd
from rdkit import Chem
from pathlib import Path

In [56]:
roche_lit_data = pd.read_csv('data/literature_rxndata.csv', encoding='unicode_escape')

In [144]:
reag_list = [col for col in roche_lit_data.columns if col.endswith('smiles')][1:-2]
reag_list

['reagent_1_smiles',
 'reagent_2_smiles',
 'reagent_3_smiles',
 'reagent_4_smiles',
 'catalyst_1_smiles',
 'catalyst_2_smiles',
 'ligand_1_smiles',
 'ligand_2_smiles',
 'additive_1_smiles',
 'solvent_1_smiles',
 'solvent_2_smiles']

In [103]:
yield_filter = (roche_lit_data['product_1_yield'].fillna(0.0))>=0.3
mono_filter = roche_lit_data['product_2_smiles'].isna()

In [77]:
roche_mono = roche_lit_data[roche_lit_data['product_2_smiles'].isna()]

In [78]:
roche_mono.drop_duplicates('product_1_smiles')

,rxn_id,source_id,rxn_type,temperature_degC,time_h,atmosphere,scale_mol,concentration_moll,stmat_1_smiles,stmat_1_eq,...,solvent_1_smiles,solvent_1_fraction,solvent_2_smiles,solvent_2_fraction,product_1_smiles,product_1_yield,product_1_yieldtype,product_2_smiles,product_2_yield,product_2_yieldtype
0,lit_pub_bo_1,10.1021/acscatal.0c00152,borylation,100,48,ar,0.00017,NaN,O1CCCC1,26.1,...,O1CCCC1,NaN,NaN,NaN,O1B(OC(C)(C)C1(C)C)C2COCC2,0.15,GC,NaN,NaN,NaN
17,lit_pub_bo_18,10.1021/ja1006405,borylation,80,24,n2,0.00100,1.0,ClC1=CC=C2SC=3C=CC=CC3NC2=C1,1.0,...,O1CCCC1,1.0,NaN,NaN,ClC1=CC=C2SC3=CC=CC(B4OC(C)(C)C(O4)(C)C)=C3NC2=C1,0.87,Isolated,NaN,NaN,NaN
18,lit_pub_bo_19,10.1021/ja1006405,borylation,80,4,n2,0.00100,1.0,C=1C=CC=2NC=CC2C1,1.0,...,O1CCCC1,1.0,NaN,NaN,O1B(OC(C)(C)C1(C)C)C2=CC=CC=3C=CNC32,0.66,Isolated,NaN,NaN,NaN
19,lit_pub_bo_20,10.1021/ja1006405,borylation,80,4,n2,0.00100,1.0,ClC=1C=CC=2NC=CC2C1,1.0,...,O1CCCC1,1.0,NaN,NaN,ClC1=CC=2C=CNC2C(=C1)B3OC(C)(C)C(O3)(C)C,0.64,Isolated,NaN,NaN,NaN
20,lit_pub_bo_21,10.1021/ja1006405,borylation,80,8,n2,0.00100,1.0,C=1C=CC2=C(C1)NC3=C2CCC3,1.0,...,O1CCCC1,1.0,NaN,NaN,O1B(OC(C)(C)C1(C)C)C2=CC=CC3=C2NC4=C3CCC4,0.64,Isolated,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296,lit_pub_bo_1297,10.1002/anie.201104544,borylation,80,17,ar,0.00050,0.5,ClC1=CC=CC(C=NN(C)C)=C1,1.0,...,O1CCCC1,1.0,NaN,NaN,ClC1=CC=C(B2OC(C)(C)C(O2)(C)C)C(C=NN(C)C)=C1,0.84,Isolated,NaN,NaN,NaN
1297,lit_pub_bo_1298,10.1002/anie.201104544,borylation,80,10,ar,0.00050,0.5,N(=CC1=CC=C(OC)C=C1)N(C)C,1.0,...,O1CCCC1,1.0,NaN,NaN,N(=CC1=CC=C(OC)C=C1B2OC(C)(C)C(O2)(C)C)N(C)C,0.77,Isolated,NaN,NaN,NaN
1298,lit_pub_bo_1299,10.1002/anie.201104544,borylation,80,8,ar,0.00050,0.5,ClC1=CC=C(C=NN(C)C)C=C1,1.0,...,O1CCCC1,1.0,NaN,NaN,ClC1=CC=C(C=NN(C)C)C(=C1)B2OC(C)(C)C(O2)(C)C,0.73,Isolated,NaN,NaN,NaN
1299,lit_pub_bo_1300,10.1002/anie.201104544,borylation,80,7,ar,0.00050,0.5,FC1=CC=C(C=NN(C)C)C=C1,1.0,...,O1CCCC1,1.0,NaN,NaN,FC1=CC=C(C=NN(C)C)C(=C1)B2OC(C)(C)C(O2)(C)C,0.70,Isolated,NaN,NaN,NaN


In [89]:
long_smi = roche_lit_data['product_1_smiles'].apply(lambda x: len(str(x))>=5)

In [160]:
roche_nearly_there = roche_lit_data[yield_filter&long_smi].drop_duplicates(['product_1_smiles'])

In [161]:
roche_nearly_there

,rxn_id,source_id,rxn_type,temperature_degC,time_h,atmosphere,scale_mol,concentration_moll,stmat_1_smiles,stmat_1_eq,...,solvent_1_smiles,solvent_1_fraction,solvent_2_smiles,solvent_2_fraction,product_1_smiles,product_1_yield,product_1_yieldtype,product_2_smiles,product_2_yield,product_2_yieldtype
1,lit_pub_bo_2,10.1021/acscatal.0c00152,borylation,100,48,ar,0.00017,NaN,O1CCCC1,26.1,...,O1CCCC1,NaN,NaN,NaN,O1B(OC(C)(C)C1(C)C)C2COCC2,0.42,GC,NaN,NaN,NaN
17,lit_pub_bo_18,10.1021/ja1006405,borylation,80,24,n2,0.00100,1.0,ClC1=CC=C2SC=3C=CC=CC3NC2=C1,1.0,...,O1CCCC1,1.0,NaN,NaN,ClC1=CC=C2SC3=CC=CC(B4OC(C)(C)C(O4)(C)C)=C3NC2=C1,0.87,Isolated,NaN,NaN,NaN
18,lit_pub_bo_19,10.1021/ja1006405,borylation,80,4,n2,0.00100,1.0,C=1C=CC=2NC=CC2C1,1.0,...,O1CCCC1,1.0,NaN,NaN,O1B(OC(C)(C)C1(C)C)C2=CC=CC=3C=CNC32,0.66,Isolated,NaN,NaN,NaN
19,lit_pub_bo_20,10.1021/ja1006405,borylation,80,4,n2,0.00100,1.0,ClC=1C=CC=2NC=CC2C1,1.0,...,O1CCCC1,1.0,NaN,NaN,ClC1=CC=2C=CNC2C(=C1)B3OC(C)(C)C(O3)(C)C,0.64,Isolated,NaN,NaN,NaN
20,lit_pub_bo_21,10.1021/ja1006405,borylation,80,8,n2,0.00100,1.0,C=1C=CC2=C(C1)NC3=C2CCC3,1.0,...,O1CCCC1,1.0,NaN,NaN,O1B(OC(C)(C)C1(C)C)C2=CC=CC3=C2NC4=C3CCC4,0.64,Isolated,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296,lit_pub_bo_1297,10.1002/anie.201104544,borylation,80,17,ar,0.00050,0.5,ClC1=CC=CC(C=NN(C)C)=C1,1.0,...,O1CCCC1,1.0,NaN,NaN,ClC1=CC=C(B2OC(C)(C)C(O2)(C)C)C(C=NN(C)C)=C1,0.84,Isolated,NaN,NaN,NaN
1297,lit_pub_bo_1298,10.1002/anie.201104544,borylation,80,10,ar,0.00050,0.5,N(=CC1=CC=C(OC)C=C1)N(C)C,1.0,...,O1CCCC1,1.0,NaN,NaN,N(=CC1=CC=C(OC)C=C1B2OC(C)(C)C(O2)(C)C)N(C)C,0.77,Isolated,NaN,NaN,NaN
1298,lit_pub_bo_1299,10.1002/anie.201104544,borylation,80,8,ar,0.00050,0.5,ClC1=CC=C(C=NN(C)C)C=C1,1.0,...,O1CCCC1,1.0,NaN,NaN,ClC1=CC=C(C=NN(C)C)C(=C1)B2OC(C)(C)C(O2)(C)C,0.73,Isolated,NaN,NaN,NaN
1299,lit_pub_bo_1300,10.1002/anie.201104544,borylation,80,7,ar,0.00050,0.5,FC1=CC=C(C=NN(C)C)C=C1,1.0,...,O1CCCC1,1.0,NaN,NaN,FC1=CC=C(C=NN(C)C)C(=C1)B2OC(C)(C)C(O2)(C)C,0.70,Isolated,NaN,NaN,NaN


In [107]:
substrate = roche_nearly_there['stmat_1_smiles']

In [154]:
def standardise_smiles(smi):
    try:
        return Chem.CanonSmiles(smi)
    except:
        return np.nan

In [162]:
smiles_cols = [col for col in roche_lit_data.columns if col.endswith('smiles')]
roche_nearly_there[smiles_cols] = roche_nearly_there[smiles_cols].applymap(standardise_smiles)

[16:27:28] Explicit valence for atom # 0 F, 2, is greater than permitted
[16:27:28] Explicit valence for atom # 0 F, 2, is greater than permitted
[16:27:28] Explicit valence for atom # 0 F, 2, is greater than permitted
[16:27:28] Explicit valence for atom # 0 F, 2, is greater than permitted
[16:27:28] Explicit valence for atom # 0 F, 2, is greater than permitted
[16:27:28] Explicit valence for atom # 0 F, 2, is greater than permitted
[16:27:28] Explicit valence for atom # 0 F, 2, is greater than permitted
[16:27:28] Explicit valence for atom # 0 F, 2, is greater than permitted
[16:27:28] Explicit valence for atom # 0 F, 2, is greater than permitted
[16:27:28] Explicit valence for atom # 0 F, 2, is greater than permitted
[16:27:28] Explicit valence for atom # 0 F, 2, is greater than permitted
[16:27:28] Explicit valence for atom # 0 F, 2, is greater than permitted
[16:27:28] Explicit valence for atom # 0 F, 2, is greater than permitted
[16:27:28] Explicit valence for atom # 0 F, 2, is g

In [163]:
for idx, row in roche_nearly_there[smiles_cols[1:-2]].iterrows():
    new_row = row.dropna()
    
    roche_nearly_there.loc[idx,'reagents'] = '.'.join(new_row)

In [164]:
roche_nearly_there['reagents']

1       CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.c1cnc2c(c...
17      CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBO...
18      CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBO...
19      CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBO...
20      CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBO...
                              ...                        
1296    CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBO...
1297    CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBO...
1298    CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBO...
1299    CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBO...
1300    CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBO...
Name: reagents, Length: 685, dtype: object

In [201]:
roche_t5_ready = roche_nearly_there[['stmat_1_smiles','reagents','product_1_smiles','product_1_yield']]
roche_t5_ready.columns = ['reactants','reagents','product','yield']
roche_t5_ready['yield'] = roche_t5_ready['yield']*100

/var/folders/5m/v_fktf9n5jg1_hwxp106mymw0000gn/T/ipykernel_89518/3330052506.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  roche_t5_ready['yield'] = roche_t5_ready['yield']*100


In [202]:
roche_t5_ready = roche_t5_ready.dropna(subset=['reactants','product'])

In [212]:
roche_t5_ready.to_csv('data/roche_t5_ready.csv')

In [207]:
for idx, (reac,reag,prod) in roche_t5_ready[['reactants','reagents','product']].iterrows():
    roche_t5_ready.loc[idx, 'reagents_source'] = reac+'>>'+prod
    roche_t5_ready.loc[idx, 'product_source'] = '.'.join([reac,reag])+'>>'
    roche_t5_ready.loc[idx, 'yield_source'] = reac+'>'+reag+'>'+prod
roche_t5_ready['reactants_source'] = roche_t5_ready['product']

In [187]:
roche_t5_ready['prod_source'].iloc[1]

'Clc1ccc2c(c1)Nc1ccccc1S2.CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBOC1(C)C.CC(C)(C)c1ccnc(-c2cc(C(C)(C)C)ccn2)c1.C1CCOC1>>'

In [213]:
np.random.seed(131)
permut = np.random.permutation(roche_t5_ready.index)
df_len = len(roche_t5_ready)
train_idx = permut[:int(df_len*0.8)]
val_idx = permut[int(df_len*0.8):int(df_len*0.9)]
test_idx = permut[int(df_len*0.9):]

In [211]:
roche_t5_ready.dropna()

,reactants,reagents,product,yield,reagents_source,product_source,reactants_source,yield_source
1,C1CCOC1,CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.c1cnc2c(c...,CC1(C)OB(C2CCOC2)OC1(C)C,42.0,C1CCOC1>>CC1(C)OB(C2CCOC2)OC1(C)C,C1CCOC1.CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.c...,CC1(C)OB(C2CCOC2)OC1(C)C,C1CCOC1>CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.c...
17,Clc1ccc2c(c1)Nc1ccccc1S2,CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBO...,CC1(C)OB(c2cccc3c2Nc2cc(Cl)ccc2S3)OC1(C)C,87.0,Clc1ccc2c(c1)Nc1ccccc1S2>>CC1(C)OB(c2cccc3c2Nc...,Clc1ccc2c(c1)Nc1ccccc1S2.CC1(C)OB(B2OC(C)(C)C(...,CC1(C)OB(c2cccc3c2Nc2cc(Cl)ccc2S3)OC1(C)C,Clc1ccc2c(c1)Nc1ccccc1S2>CC1(C)OB(B2OC(C)(C)C(...
18,c1ccc2[nH]ccc2c1,CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBO...,CC1(C)OB(c2cccc3cc[nH]c23)OC1(C)C,66.0,c1ccc2[nH]ccc2c1>>CC1(C)OB(c2cccc3cc[nH]c23)OC...,c1ccc2[nH]ccc2c1.CC1(C)OB(B2OC(C)(C)C(C)(C)O2)...,CC1(C)OB(c2cccc3cc[nH]c23)OC1(C)C,c1ccc2[nH]ccc2c1>CC1(C)OB(B2OC(C)(C)C(C)(C)O2)...
19,Clc1ccc2[nH]ccc2c1,CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBO...,CC1(C)OB(c2cc(Cl)cc3cc[nH]c23)OC1(C)C,64.0,Clc1ccc2[nH]ccc2c1>>CC1(C)OB(c2cc(Cl)cc3cc[nH]...,Clc1ccc2[nH]ccc2c1.CC1(C)OB(B2OC(C)(C)C(C)(C)O...,CC1(C)OB(c2cc(Cl)cc3cc[nH]c23)OC1(C)C,Clc1ccc2[nH]ccc2c1>CC1(C)OB(B2OC(C)(C)C(C)(C)O...
20,c1ccc2c3c([nH]c2c1)CCC3,CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBO...,CC1(C)OB(c2cccc3c4c([nH]c23)CCC4)OC1(C)C,64.0,c1ccc2c3c([nH]c2c1)CCC3>>CC1(C)OB(c2cccc3c4c([...,c1ccc2c3c([nH]c2c1)CCC3.CC1(C)OB(B2OC(C)(C)C(C...,CC1(C)OB(c2cccc3c4c([nH]c23)CCC4)OC1(C)C,c1ccc2c3c([nH]c2c1)CCC3>CC1(C)OB(B2OC(C)(C)C(C...
...,...,...,...,...,...,...,...,...
1296,CN(C)N=Cc1cccc(Cl)c1,CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBO...,CN(C)N=Cc1cc(Cl)ccc1B1OC(C)(C)C(C)(C)O1,84.0,CN(C)N=Cc1cccc(Cl)c1>>CN(C)N=Cc1cc(Cl)ccc1B1OC...,CN(C)N=Cc1cccc(Cl)c1.CC1(C)OB(B2OC(C)(C)C(C)(C...,CN(C)N=Cc1cc(Cl)ccc1B1OC(C)(C)C(C)(C)O1,CN(C)N=Cc1cccc(Cl)c1>CC1(C)OB(B2OC(C)(C)C(C)(C...
1297,COc1ccc(C=NN(C)C)cc1,CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBO...,COc1ccc(C=NN(C)C)c(B2OC(C)(C)C(C)(C)O2)c1,77.0,COc1ccc(C=NN(C)C)cc1>>COc1ccc(C=NN(C)C)c(B2OC(...,COc1ccc(C=NN(C)C)cc1.CC1(C)OB(B2OC(C)(C)C(C)(C...,COc1ccc(C=NN(C)C)c(B2OC(C)(C)C(C)(C)O2)c1,COc1ccc(C=NN(C)C)cc1>CC1(C)OB(B2OC(C)(C)C(C)(C...
1298,CN(C)N=Cc1ccc(Cl)cc1,CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBO...,CN(C)N=Cc1ccc(Cl)cc1B1OC(C)(C)C(C)(C)O1,73.0,CN(C)N=Cc1ccc(Cl)cc1>>CN(C)N=Cc1ccc(Cl)cc1B1OC...,CN(C)N=Cc1ccc(Cl)cc1.CC1(C)OB(B2OC(C)(C)C(C)(C...,CN(C)N=Cc1ccc(Cl)cc1B1OC(C)(C)C(C)(C)O1,CN(C)N=Cc1ccc(Cl)cc1>CC1(C)OB(B2OC(C)(C)C(C)(C...
1299,CN(C)N=Cc1ccc(F)cc1,CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.CC1(C)OBO...,CN(C)N=Cc1ccc(F)cc1B1OC(C)(C)C(C)(C)O1,70.0,CN(C)N=Cc1ccc(F)cc1>>CN(C)N=Cc1ccc(F)cc1B1OC(C...,CN(C)N=Cc1ccc(F)cc1.CC1(C)OB(B2OC(C)(C)C(C)(C)...,CN(C)N=Cc1ccc(F)cc1B1OC(C)(C)C(C)(C)O1,CN(C)N=Cc1ccc(F)cc1>CC1(C)OB(B2OC(C)(C)C(C)(C)...


In [209]:
roche_folder = Path('data/roche')
roche_folder.mkdir(parents=True, exist_ok=True)
for task in ['reactants','reagents','product','yield']:
    task_path = (roche_folder/task)
    task_path.mkdir(parents=True, exist_ok=True)
    for slice, label in zip([train_idx,val_idx,test_idx],['train','val','test']):
        frame = roche_t5_ready.reindex(slice)
        frame[f'{task}_source'].to_csv(path_or_buf=f'{task_path}/{label}.source',header=False,index=False)
        frame[f'{task}'].to_csv(path_or_buf=f'{task_path}/{label}.target',header=False,index=False)
task_path = (roche_folder/'mixed')
task_path.mkdir(parents=True, exist_ok=True)
for task in ['reactants','reagents','product']:
    for slice, label in zip([train_idx,val_idx,test_idx],['train','val','test']):
        frame = roche_t5_ready.reindex(slice)
        src = task.capitalize()+':'+frame[f'{task}_source']
        src.to_csv(path_or_buf=f'{task_path}/{label}.source',header=False,index=False,mode='a')
        frame[f'{task}'].to_csv(path_or_buf=f'{task_path}/{label}.target',header=False,index=False,mode='a')


In [225]:
rs = 891
np.random.seed(rs)
permut = np.random.permutation(roche_t5_ready.index)
df_len = len(roche_t5_ready)
train_idx = permut[:int(df_len*0.8)]
val_idx = permut[int(df_len*0.8):int(df_len*0.9)]
test_idx = permut[int(df_len*0.9):]
roche_folder = Path(f'data/roche_{rs}')
roche_folder.mkdir(parents=True, exist_ok=True)
for task in ['reactants','reagents','product','yield']:
    task_path = (roche_folder/task)
    task_path.mkdir(parents=True, exist_ok=True)
    for slice, label in zip([train_idx,val_idx,test_idx],['train','val','test']):
        frame = roche_t5_ready.reindex(slice)
        frame[f'{task}_source'].to_csv(path_or_buf=f'{task_path}/{label}.source',header=False,index=False)
        frame[f'{task}'].to_csv(path_or_buf=f'{task_path}/{label}.target',header=False,index=False)
task_path = (roche_folder/'mixed')
task_path.mkdir(parents=True, exist_ok=True)
for task in ['reactants','reagents','product']:
    for slice, label in zip([train_idx,val_idx,test_idx],['train','val','test']):
        frame = roche_t5_ready.reindex(slice)
        src = task.capitalize()+':'+frame[f'{task}_source']
        src.to_csv(path_or_buf=f'{task_path}/{label}.source',header=False,index=False,mode='a')
        frame[f'{task}'].to_csv(path_or_buf=f'{task_path}/{label}.target',header=False,index=False,mode='a')